In [1]:
import pandas as pd
import numpy as np
import requests
import json
from time import sleep
from bs4 import BeautifulSoup
import sqlite3

In [92]:
conn = sqlite3.connect('housing.db')
cur = conn.cursor()
cur.execute('''CREATE TABLE apartments (
    apt_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    address TEXT NOT NULL,
    unit TEXT NOT NULL,
    sqft TEXT NOT NULL,
    bed TEXT NOT NULL,
    bath TEXT NOT NULL,
    price TEXT NOT NULL,
    city TEXT NOT NULL,
    state TEXT NOT NULL,
    zip_code INTEGER NOT NULL,
    description TEXT NOT NULL,
    details TEXT NOT NULL,
    url TEXT NOT NULL,
    date NUMERIC NOT NULL
);''')
conn.commit()
conn.close()

In [91]:
conn = sqlite3.connect('housing.db')
cur = conn.cursor()

cur.execute('DROP TABLE apartments')

selected_data = cur.fetchall()

conn.commit()

conn.close()

In [14]:
from lxml import html
import requests
import unicodecsv as csv
import argparse
import json


def clean(text):
    if text:
        return ' '.join(' '.join(text).split())
    return None


def get_headers():
    # Creating headers.
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    return headers


def create_url(zipcode, filter):
    # Creating Zillow URL based on the filter.

    if filter == "newest":
        url = "https://www.zillow.com/homes/for_sale/{0}/0_singlestory/days_sort".format(zipcode)
    elif filter == "cheapest":
        url = "https://www.zillow.com/homes/for_sale/{0}/0_singlestory/pricea_sort/".format(zipcode)
    else:
        url = "https://www.zillow.com/homes/for_sale/{0}_rb/?fromHomePage=true&shouldFireSellPageImplicitClaimGA=false&fromHomePageTab=buy".format(zipcode)
    print(url)
    return url


def save_to_file(response):
    # saving response to `response.html`

    with open("response.html", 'w') as fp:
        fp.write(response.text)


def write_data_to_csv(data):
    # saving scraped data to csv.

    with open("properties-%s.csv" % (zipcode), 'wb') as csvfile:
        fieldnames = ['title', 'address', 'city', 'state', 'postal_code', 'price', 'facts and features', 'real estate provider', 'url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)


def get_response(url):
    # Getting response from zillow.com.

    for i in range(5):
        response = requests.get(url, headers=get_headers())
        print("status code received:", response.status_code)
        if response.status_code != 200:
            # saving response to file for debugging purpose.
            save_to_file(response)
            continue
        else:
            save_to_file(response)
            return response
    return None

def get_data_from_json(raw_json_data):
    # getting data from json (type 2 of their A/B testing page)

    cleaned_data = clean(raw_json_data).replace('<!--', "").replace("-->", "")
    properties_list = []

    try:
        json_data = json.loads(cleaned_data)
        search_results = json_data.get('searchResults').get('listResults', [])

        for properties in search_results:
            address = properties.get('addressWithZip')
            property_info = properties.get('hdpData', {}).get('homeInfo')
            city = property_info.get('city')
            state = property_info.get('state')
            postal_code = property_info.get('zipcode')
            price = properties.get('price')
            bedrooms = properties.get('beds')
            bathrooms = properties.get('baths')
            area = properties.get('area')
            info = f'{bedrooms} bds, {bathrooms} ba ,{area} sqft'
            broker = properties.get('brokerName')
            property_url = properties.get('detailUrl')
            title = properties.get('statusText')

            data = {'address': address,
                    'city': city,
                    'state': state,
                    'postal_code': postal_code,
                    'price': price,
                    'facts and features': info,
                    'real estate provider': broker,
                    'url': property_url,
                    'title': title}
            properties_list.append(data)

        return properties_list

    except ValueError:
        print("Invalid json")
        return None


def parse(zipcode, filter=None):
    url = create_url(zipcode, filter)
    response = get_response(url)

    if not response:
        print("Failed to fetch the page, please check `response.html` to see the response received from zillow.com.")
        return None

    parser = html.fromstring(response.text)
    search_results = parser.xpath("//div[@id='search-results']//article")

    if not search_results:
        print("parsing from json data")
        # identified as type 2 page
        raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
        return get_data_from_json(raw_json_data)

    print("parsing from html page")
    properties_list = []
    for properties in search_results:
        raw_address = properties.xpath(".//span[@itemprop='address']//span[@itemprop='streetAddress']//text()")
        raw_city = properties.xpath(".//span[@itemprop='address']//span[@itemprop='addressLocality']//text()")
        raw_state = properties.xpath(".//span[@itemprop='address']//span[@itemprop='addressRegion']//text()")
        raw_postal_code = properties.xpath(".//span[@itemprop='address']//span[@itemprop='postalCode']//text()")
        raw_price = properties.xpath(".//span[@class='zsg-photo-card-price']//text()")
        raw_info = properties.xpath(".//span[@class='zsg-photo-card-info']//text()")
        raw_broker_name = properties.xpath(".//span[@class='zsg-photo-card-broker-name']//text()")
        url = properties.xpath(".//a[contains(@class,'overlay-link')]/@href")
        raw_title = properties.xpath(".//h4//text()")

        address = clean(raw_address)
        city = clean(raw_city)
        state = clean(raw_state)
        postal_code = clean(raw_postal_code)
        price = clean(raw_price)
        info = clean(raw_info).replace(u"\xb7", ',')
        broker = clean(raw_broker_name)
        title = clean(raw_title)
        property_url = "https://www.zillow.com" + url[0] if url else None
        is_forsale = properties.xpath('.//span[@class="zsg-icon-for-sale"]')

        properties = {'address': address,
                      'city': city,
                      'state': state,
                      'postal_code': postal_code,
                      'price': price,
                      'facts and features': info,
                      'real estate provider': broker,
                      'url': property_url,
                      'title': title}
        if is_forsale:
            properties_list.append(properties)
    return properties_list


if __name__ == "__main__":
    # Reading arguments

    argparser = argparse.ArgumentParser(formatter_class=argparse.RawTextHelpFormatter)
    argparser.add_argument('zipcode', help='')
    sortorder_help = """
    available sort orders are :
    newest : Latest property details,
    cheapest : Properties with cheapest price
    """

    argparser.add_argument('sort', nargs='?', help=sortorder_help, default='Homes For You')
    args = argparser.parse_args()
    zipcode = args.zipcode
    sort = args.sort
    print ("Fetching data for %s" % (zipcode))
    scraped_data = parse(zipcode, sort)
    if scraped_data:
        print ("Writing data to output file")
        write_data_to_csv(scraped_data)

In [4]:
headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

base_url = 'https://www.trulia.com'
page_url = '/for_rent/Austin,TX/'

In [93]:
def get_url_list(base_url, page_url):

    url_list = []
    
    for i in range(6,8):
#     while (last_page = False):
        
        response = requests.get(base_url+page_url, headers=headers)
        print("status code received:", response.status_code)
        
        if response.status_code != 200:
            print("Failed: ", response.status_code)
        else:
            soup = BeautifulSoup(response.content, 'lxml')


        

        container = soup.find('div', {'data-testid': 'search-result-list-container'})

        for div in soup.find_all('div', {'data-hero-element-id': 'srp-home-card', 'data-hero-element-id':'false'}):

            url = div.find('a').attrs['href']
            url_list.append(url)

        ## Check if last page and exit while loop
            
        if soup.find('a', {'aria-label': 'Next Page'}):
            last_page = False
            page_url = soup.find('a', {'aria-label': 'Next Page'})['href']
            print("URL for next page: " + base_url+page_url)
            print('Sleeping half second')
            sleep(.5)
        else:

            last_page = True
            print("No more pages")

    return url_list

In [94]:
url_list = get_url_list(base_url, page_url)

status code received: 200
URL for next page: https://www.trulia.com/for_rent/Austin,TX/2_p/
Sleeping half second
status code received: 200
URL for next page: https://www.trulia.com/for_rent/Austin,TX/3_p/
Sleeping half second


In [95]:
def get_apartment_data(base_url, current_url):

    response = requests.get(base_url+current_url, headers=headers)
    if response.status_code != 200:
        print("Failed: ", response.status_code)
    else:
        soup = BeautifulSoup(response.content, 'lxml')

    apartment_list = []

    for floor_plan_table in soup.find_all('table', {'data-testid': 'floor-plan-group'}):
        for tr in floor_plan_table.find_all('tr'):

            unit = tr.find('div', {'color': 'highlight'}).text

            sqft = tr.find('td', {'class': 'FloorPlanTable__FloorPlanFloorSpaceCell-sc-1ghu3y7-5'}).text
#             sqft = int(sqft.replace('sqft', '').replace(',',''))
            
            bed = tr.find_all('td', {'class': 'FloorPlanTable__FloorPlanFeaturesCell-sc-1ghu3y7-4'})[0].text
#             bed = float(bed.replace('bd', ''))
            
            bath = tr.find_all('td', {'class': 'FloorPlanTable__FloorPlanFeaturesCell-sc-1ghu3y7-4'})[1].text
#             bath = float(bath.replace('ba', ''))
            
            price = tr.find_all('td', {'class': 'FloorPlanTable__FloorPlanCell-sc-1ghu3y7-2', 
                                       'class': 'FloorPlanTable__FloorPlanSMCell-sc-1ghu3y7-8'},
                                        limit=2)[1].text
            price = price.replace('Contact', '')

            name = soup.find('span', {'data-testid': 'home-details-summary-headline'}).text

            address = soup.find_all('span', {'data-testid': 'home-details-summary-city-state'})[0].text

            city_state_zip = soup.find_all('span', {'data-testid': 'home-details-summary-city-state'})[1].text
            city, state, zip_code = city_state_zip.replace(',', '').split()
            zip_code = int(zip_code)
            
            description = soup.find('div', {'data-testid': 'home-description-text-description-text'}).text

            details = [detail.text for detail in soup.find_all(
                                                        'li', {'class': 'FeatureList__FeatureListItem-iipbki-0 dArMue'}
                                                            )]
            details = ' ,'.join(details)
            
            apartment_url = base_url + current_url
            
            apartment_list.append([name, address, unit, sqft, bed, bath, price, city, state, zip_code, description, details, apartment_url])

    return apartment_list


In [96]:
len(url_list)

44

In [97]:
apts_data = []
for current_url in url_list[:10]:
    apts_data.extend(get_apartment_data(base_url, current_url))


status code received: 200
URL: https://www.trulia.com/c/tx/austin/8400-beech-dr-8400-beech-dr-austin-tx-78758--2072303310
status code received: 200
URL: https://www.trulia.com/c/tx/austin/the-sofia-2201-willow-creek-dr-austin-tx-78741--2072200429
status code received: 200
URL: https://www.trulia.com/c/tx/austin/park-at-summer-s-grove-2900-century-park-blvd-austin-tx-78727--2072135724
status code received: 200
URL: https://www.trulia.com/c/tx/austin/alister-apartments-1845-burton-dr-austin-tx-78741--2072199610
status code received: 200
URL: https://www.trulia.com/p/tx/austin/1614-wheless-ln-102-austin-tx-78723--2412871691
status code received: 200
URL: https://www.trulia.com/c/tx/austin/the-oaks-130-cumberland-rd-austin-tx-78704--2072090465
status code received: 200
URL: https://www.trulia.com/c/tx/austin/mueller-flats-apartments-1071-clayton-ln-austin-tx-78723--2072113721
status code received: 200
URL: https://www.trulia.com/c/tx/austin/val-dor-flats-1521-reagan-hill-dr-austin-tx-78752

In [98]:
len(apts_data)

78

In [ ]:
def sql_connection():
 
    try:
 
        conn = sqlite3.connect('housing.db')
 
        print("Connection successful")
 
    except Error:
 
        print(Error)
 
    finally:
 
        conn.close()

In [99]:
conn = sqlite3.connect('housing.db')
cur = conn.cursor()

cur.executemany('''INSERT INTO apartments (name, address, unit, sqft, 
                                           bed, bath, price, city, state, 
                                           zip_code, description, details, url, date) 
                   VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, date('now', 'localtime'))''', apts_data)
conn.commit()
conn.close()

In [73]:
conn = sqlite3.connect('housing.db')
cur = conn.cursor()

cur.execute('DELETE FROM apartments')

selected_data = cur.fetchall()

conn.commit()

conn.close()

In [102]:
conn = sqlite3.connect('housing.db')
cur = conn.cursor()

cur.execute('SELECT apt_id, name, address, sqft, price, date FROM apartments')

selected_data = cur.fetchall()

conn.commit()

conn.close()

In [103]:
selected_data

[(1,
  '8400 Beech DR Apartments',
  '8400 Beech Dr',
  '525 sqft',
  '$850',
  '2020-01-08'),
 (2,
  '8400 Beech DR Apartments',
  '8400 Beech Dr',
  '525 sqft',
  '$850',
  '2020-01-08'),
 (3,
  '8400 Beech DR Apartments',
  '8400 Beech Dr',
  '1,050 sqft',
  '$1,050',
  '2020-01-08'),
 (4,
  '8400 Beech DR Apartments',
  '8400 Beech Dr',
  '1,050 sqft',
  '$1,050',
  '2020-01-08'),
 (5,
  'The Sofia Apartments',
  '2201 Willow Creek Dr',
  '554 sqft',
  '$925',
  '2020-01-08'),
 (6,
  'The Sofia Apartments',
  '2201 Willow Creek Dr',
  '554 sqft',
  '$925',
  '2020-01-08'),
 (7,
  'The Sofia Apartments',
  '2201 Willow Creek Dr',
  '554 sqft',
  '$925',
  '2020-01-08'),
 (8,
  'The Sofia Apartments',
  '2201 Willow Creek Dr',
  '554 sqft',
  '$925',
  '2020-01-08'),
 (9,
  'The Sofia Apartments',
  '2201 Willow Creek Dr',
  '554 sqft',
  '$925',
  '2020-01-08'),
 (10,
  "Park at Summer's Grove Apartments",
  '2900 Century Park Blvd',
  '1,117 sqft',
  '$1,050',
  '2020-01-08'),
 (11